In [ ]:
from utils import *
import pandas as pd
from io import StringIO
import os
import requests
import requests
from bs4 import BeautifulSoup as soup
from typing import List
from typing import Dict
import numpy as np
import unicodedata
import basedosdados as bd


In [ ]:
#municipio
dir_path  = r'C:\Users\gabri\OneDrive\vida_profissional\Projetos\base_dos_dados\br_mcid_snis\input\municipio'
files = os.listdir(dir_path)

lista_dfs = []

print('baixando dicionario da tabela de arquitetura para renomear colunas')
dicionario = change_column_name(
    url_architecture = 'https://docs.google.com/spreadsheets/d/1MNToVZvMoAgpB51cH5042DYVxiHwEDNz/edit#gid=1217233611'
)

print('baixando tipos de dados da tabela de arquitetura para importar dados')
dicionario_dtypes = change_dtypes(
    url_architecture='https://docs.google.com/spreadsheets/d/1MNToVZvMoAgpB51cH5042DYVxiHwEDNz/edit#gid=1217233611'
)

print('baixando diretorios de municípios da BD')
municipio = bd.read_table(
    dataset_id= 'br_bd_diretorios_brasil',
    table_id= 'municipio',
    billing_project_id= "pisagab-staging"
    )
municipio_dict = dict(zip(municipio.id_municipio_6, municipio.id_municipio))


for file in files:
    
    print(f'lendo arquivo {file}')
    df = pd.read_csv(
    dir_path + '\\' + file,
    sep=';',
    encoding= 'latin-1',
    decimal=',',
    thousands='.',
    dtype= dicionario_dtypes,
    )

    print('renomeando colunas')
    #rename
    df.rename(
        columns=dicionario, 
        inplace=True
    )

    print('adicionando id_municipio 7 dígitos')
    #add id_municipio 7 digits
    df['id_municipio'] = df['id_municipio'].map(municipio_dict)
    
    #check if there is any null value
    if df['id_municipio'].isna().sum() != 0:
        raise ValueError('id_municipio com valores nulos')
    
   
    print('dropando colunas')
    #delete cols
    df.drop(
        columns=[
        'Código do IBGE', 
        'Município',
        'Prestadores',
        'Serviços',
        'Natureza Jurídica',
        ], 
        inplace=True)
    
    print('tirando . de ints por precaução')
    
    strings_to_int = df.select_dtypes(include=['object']).columns
    
    for col in strings_to_int:
        df[col] = df[col].str.replace('.', '')
        print(f'col {col} formatada, . retirados')
    
    
    print('ordenando colunas')
    #redorder
    df = df[dicionario.values()]

    lista_dfs.append(df)

df = pd.concat(lista_dfs)

df.to_csv(
    r'C:\Users\gabri\OneDrive\vida_profissional\Projetos\base_dos_dados\br_mcid_snis\output\municipio.csv',
    index=False,
    sep=',',
    encoding='utf-8',
    na_rep='',
)

In [ ]:
#prestador
dir_path  = r'C:\Users\gabri\OneDrive\vida_profissional\Projetos\base_dos_dados\br_mcid_snis\input\prestador'
files = os.listdir(dir_path)

lista_dfs = []

print('baixando dicionario da tabela de arquitetura para renomear colunas')
dicionario = change_column_name(
    url_architecture = 'https://docs.google.com/spreadsheets/d/1LJIiGiw27DCvEGTjjlA2tYWN5UaN5-zETepQAcoMSgY/edit#gid=0'
)

print('baixando tipos de dados da tabela de arquitetura para importar dados')
dicionario_dtypes = change_dtypes(
    url_architecture='https://docs.google.com/spreadsheets/d/1LJIiGiw27DCvEGTjjlA2tYWN5UaN5-zETepQAcoMSgY/edit#gid=0'
)

print('baixando diretorios de municípios da BD')
municipio = bd.read_table(
    dataset_id= 'br_bd_diretorios_brasil',
    table_id= 'municipio',
    billing_project_id= "pisagab-staging"
    )

municipio_dict = dict(zip(municipio.id_municipio_6, municipio.id_municipio))


for file in files:
    
    print(f'lendo arquivo {file}')
    df = pd.read_csv(
    dir_path + '\\' + file,
    sep=';',
    encoding= 'latin-1',
    decimal=',',
    thousands='.',
    dtype= dicionario_dtypes,
    )

    print('renomeando colunas')
    #rename
    df.rename(
        columns=dicionario, 
        inplace=True
    )

    print('adicionando id_municipio 7 dígitos')
    #add id_municipio 7 digits
    df['id_municipio'] = df['id_municipio'].map(municipio_dict)
    
    #check if there is any null value
    if df['id_municipio'].isna().sum() != 0:
        raise ValueError('id_municipio com valores nulos')
    
    print('dropando colunas')
    #delete cols
    df.drop(
        columns=[
        'Município',
        ], 
        inplace=True)
   
    print('tirando . de ints por precaução')
    strings_to_int = df.select_dtypes(include=['object']).columns
    
    for col in strings_to_int:
        df[col] = df[col].str.replace('.', '')
        print(f'col {col} formatada, . retirados')
    
    
    print('removendo acentos')
    latin_list = [
        'prestador',
        'natureza_juridica',
        'tipo_servico',
        'local_atendimento_agua',
        'local_atendimento_esgoto',
    ]

    for col in latin_list:
        df[col] = df[col].apply(remove_accents)


    #insert id_natureza_juridica

    print('ordenando colunas')
    #redorder
    df = df[dicionario.values()]

    lista_dfs.append(df)

df = pd.concat(lista_dfs)

df.to_csv(
    r'C:\Users\gabri\OneDrive\vida_profissional\Projetos\base_dos_dados\br_mcid_snis\output\prestador.csv',
    index=False,
    sep=',',
    encoding='utf-8',
    na_rep='',
)

In [10]:
df['natureza_juridica'].value_counts()

dasd = {
    'Sociedade de economia mista com administração publica' : ,
    'Administracao publica direta' : ,
    'Autarquia' : ,
    'Empresa Privada' : ,
    'Empresa Pública' : '2011', 
    'Organizacao social' : '3301', 
    'Sociedade de economia mista com administracao privada' : '2038',
    'Sociedade de economia mista com administracao publica' : '2038',
}

Sociedade de economia mista com administração pública    19242
Administração pública direta                              5572
Autarquia                                                 2612
Empresa privada                                           1026
Empresa pública                                            411
Organização social                                          21
Name: natureza_juridica, dtype: int64